In [ ]:
import sys, os
sys.path.append('/home/fvalle/phd/stopwords/code/src')
from scipy.sparse import csr_matrix
import numpy as np
import pandas as pd
from filter_words import nwd_H_shuffle
import matplotlib.pyplot as plt

In [ ]:
os.chdir("/home/fvalle/phd/datasets/merged/")

In [ ]:
df_table = pd.read_csv("mainTable_all.csv", index_col=0)
df_files = pd.read_csv("files.dat", index_col=0)
df_genes = pd.read_csv("meanVariances.csv", index_col=0)

In [ ]:
df_table = df_table.loc[:,df_table.columns[np.random.randint(len(df_table.columns), size=100)]]

In [ ]:
df_table = df_table.where(df_table > 10, 0).astype(int)
df_table = df_table[~(df_table.T==0).all()]

In [ ]:
n_wd_csr = csr_matrix(df_table)
n_wd_csr

In [ ]:
result_H = nwd_H_shuffle(n_wd_csr, N_s=10)

In [ ]:
import pickle
with open("result_H.pkl", 'wb') as f:
    pickle.dump(result_H, f)

In [ ]:
## make dataframe
df=pd.DataFrame(index = sorted(list(df_table.index)) )
df['H'] = result_H['H-emp']
df['H-tilde'] =  result_H['H-null-mu']
df['H-tilde_std'] =  result_H['H-null-std']
df['N'] = np.array(n_wd_csr.sum(axis=1))[:,0]

In [ ]:
arr_N = np.array(df['N'])
arr_H = np.array(df['H'])
arr_H_tilde = np.array(df['H-tilde'])
arr_H_tilde_std = np.array(df['H-tilde_std'])

list_w = list(df.index)

In [ ]:
fig, ax=plt.subplots(figsize=(10,10))
D = len(df_table.columns) ## Number of documents to get maximum possible value for entropy
Dmax = np.log2(D)
ax.plot([np.min(arr_N),np.max(arr_N)], [Dmax,Dmax],  lw=1, ls=':',  color='black')
x = arr_N
y = arr_H
ax.plot(x, y, ms=5, lw=0, marker='o', color='blue', alpha=0.5, rasterized=1,zorder=1)
ind_sort = np.argsort(arr_N)
x = arr_N[ind_sort]
y = arr_H_tilde[ind_sort]
yerr = arr_H_tilde_std[ind_sort]*5
ax.errorbar(x, y, yerr=yerr, color='orange', lw=1, alpha=0.9, rasterized=1,zorder=2)
ax.set_xlabel('Word frequency $n(w)$', fontsize=20)
ax.set_ylabel('$H(w|C)$', fontsize=20)
ax.set_xscale('log')

hk = pd.read_csv("/home/fvalle/phd/datasets/tcga/hk/HK_exons.csv", index_col=0).index.unique()
hk

list_w_sel = hk[hk.isin(df_table.index)].values[:] ## words we mark
for i_sel, w_sel in enumerate(list_w_sel):
    iw = list_w.index(w_sel)
    x = arr_N[iw]
    y = arr_H[iw]
    ax.scatter(x, y, marker='x', facecolors='green',zorder=3)

    x = arr_N[iw]
    y = arr_H[iw]
    ax.text(x,y, w_sel, color='green')

plt.show()
fig.savefig("H(w|c)_housekeeping.pdf")